# Cayley-Dickson Construction Applied to Qi Definition

*Version 1*

My original implementation of Gaussian integers included two classes, ``Zi`` and ``Qi``, where, for example, ``Zi(2, -7)`` represents a Gaussian integer, and ``Qi(-2/3, 4/5)`` represents a Gaussian rational.

I'd like to extend this code to include rational-valued quaternions and octonions using the Cayley-Dickson construction.

The [Cayley-Dickson construction](https://en.wikipedia.org/wiki/Cayley%E2%80%93Dickson_construction) is a process by which one can use a recursive definition of conjugation together with a recursive definition of multiplication to use...
* pairs of real numbers ($\mathbb{R}$) to create complex numbers,
* pairs of complex numbers ($\mathbb{C}$) to create quaternions,
* pairs of quaternions ($\mathbb{H}$) to create octonions,
* pairs of octonions ($\mathbb{O}$) to create sedenions ($\mathbb{S}$), and so on.

For more specifics, see my write-up about the Cayley-Dickson construction [at this link](https://abstract-algebra.readthedocs.io/en/latest/55_cayley_dickson.html).

In [1]:
from cayley_dickson_integers import Zi, SetScalarMult
from random import randint, seed
from fractions import Fraction
from functools import wraps

In [2]:
def gaussian_rational(fnc):
    """For use as a property that casts an argument into Gaussian rational."""
    @wraps(fnc)
    def gaussian_rational_wrapper(arg, num):
        qi = to_gaussian_rational(num)
        return fnc(arg, qi)
    return gaussian_rational_wrapper

In [15]:
class BasePair:
    
    def __init__(self, re=None, im=None):
        self.re = re
        self.im = im
    
    @property
    def real(self):
        return self.re

    @property
    def imag(self):
        return self.im

    def __eq__(self, other):
        return (self.re == other.re) and (self.im == other.im)

    def __ne__(self, other):
        return (self.re != other.re) or (self.im != other.im)

    def __len__(self):
        return 2

    def __getitem__(self, index):
        if isinstance(index, int):
            if index == 0:
                return self.re
            elif index == 1:
                return self.im
            else:
                raise IndexError(f"Index {index} out of range")
        else:
            raise TypeError(f"Index {index} must be a non-negative integer")

    def __iter__(self):
        return iter((self.re, self.im))  

In [16]:
class Qi(BasePair):
    """Gaussian Rational Number Class"""

    __MAX_DENOMINATOR = 1_000_000

    def __init__(self, re=None, im=None):
        
        super().__init__(re, im)

        # CLAUSE AA --------------------------------------------------------
        # re is a str, float, int, or Fraction; and
        # im is a str, float, int, Fraction, or None

        if isinstance(re, (str, float, int, Fraction)):
            if isinstance(re, Fraction):
                self.__re = re
            else:
                self.__re = Fraction(re)
            if im is None:
                self.__im = Fraction(0)
            elif isinstance(im, (str, float, int, Fraction)):
                if isinstance(im, Fraction):
                    self.__im = im
                else:
                    self.__im = Fraction(im)
            else:
                raise Exception(f"Inputs incompatible: {re} and {im}")

        # CLAUSE BA --------------------------------------------------------
        # re is a complex; and
        # im is complex, a Qi, or None

        elif isinstance(re, complex):
            if im is None:
                self.__re = Fraction(re.real)
                self.__im = Fraction(re.imag)
            elif isinstance(im, (Qi, complex)):
                self.__re = Qi(re.real, re.imag)
                self.__im = Qi(im.real, im.imag)
            else:
                raise Exception(f"Inputs incompatible: {re} and {im}")

        # CLAUSE CA --------------------------------------------------------
        # re is a Qi, and im is complex, a Qi, or None

        elif isinstance(re, Qi):
            if im is None:
                self.__re = re.real
                self.__im = re.imag
            elif isinstance(im, (complex, Qi)):
                self.__re = Qi(re)
                self.__im = Qi(im)
            else:
                raise Exception(f"Inputs incompatible: {re} and {im}")

        # CLAUSE DA --------------------------------------------------------
        # re is a Zi, and im is a Zi, or None

        elif isinstance(re, Zi):
            if im is None:
                if re.is_complex():
                    self.__re = Fraction(re.real)
                    self.__im = Fraction(re.imag)
                else:
                    self.__re = Qi(re.real)
                    self.__im = Qi(re.imag)
            elif isinstance(im, Zi) and im.order() == re.order():
                    self.__re = Qi(re)
                    self.__im = Qi(im)
            else:
                raise Exception(f"Inputs incompatible: {re} and {im}")

        # CLAUSE EA --------------------------------------------------------
        # re is a list or tuple of numbers with length equal to a
        # power of 2, and im is None, or it is a tuple or list
        # similar to the one input for re.
        elif isinstance(re, (tuple, list)):
            z = Zi.from_array(re)
            if im is None:
                self.__re = z.real
                self.__im = z.imag
            elif isinstance(im, (tuple, list)) and len(im) == len(re):
                w = Zi.from_array(im)
                self.__re = z
                self.__im = w
            else:
                raise Exception(f"Inputs incompatible: {re} and {im}")

        # CLAUSE FA --------------------------------------------------------
        # Both re and im are None

        elif re is None:
            self.__re = 0
            if im is None:
                self.__im = 0
            else:
                raise Exception(f"If re is None, then im must be None. But im = {im}")
        else:
            raise Exception(f"Unexpected combination of input types: {re} and {im}")

    @classmethod
    def max_denominator(cls, denom=None):
        if denom is None:
            return cls.__MAX_DENOMINATOR
        elif isinstance(denom, int) and denom > 1:
            cls.__MAX_DENOMINATOR = denom
            return cls.__MAX_DENOMINATOR
        else:
            raise ValueError(f"Maximum denominator, {denom}, must be an integer > 1")

    def __repr__(self):
        """This works! DO NOT MESS WITH IT."""
        re, im = self
        if isinstance(re, Fraction) and isinstance(im, Fraction):
            return f"{self.__class__.__name__}({repr(str(re))}, {repr(str(im))})"
        else:
            return f"{self.__class__.__name__}({repr(re)}, {repr(im)})"
    
    def __str__(self):
        re, im = self
        if isinstance(re, Fraction) and isinstance(im, Fraction):
            if im < 0:
                return f"({re}{im}j)"
            else:
                return f"({re}+{im}j)"
        else:
            return "NOT SUPPORTED"

    def __neg__(self):
        """Negate this Qi."""
        return Qi(-self.__re, -self.__im)

    @gaussian_rational
    def __add__(self, other):
        return Qi(self.__re + other.real, self.__im + other.imag)
    
    def __sub__(self, other):
        return Zi(self.__re - other.real, self.__im - other.imag)

## Generate Unittests

The following code is used to write a draft version of unit tests for the Qi class **init** method.

The output is just a fist-cut at a complete coverage set, because not all the printed unit tests will be correct, or even run, so careful checking and editing of the output is required.

In [17]:
import itertools

def print_Qi_unittests(inputs1, inputs2):
    """Print a draft version of unit tests for the Qi class init method.
    The output must be carefully checked and corrected, if necessary.
    Also, there may be exceptions where input cases are not yet supported.
    Look for the text '<<< ERROR >>>'.

    input1 and input2 should be lists of (key, value) tuples, where each
    key is a string to be used in a comment line indicating the value's type.
    """
    for x, y in itertools.product(inputs1, inputs2):
        typ1 = x[0]; typ2 = y[0]
        val1 = x[1]; val2 = y[1]
        print(f"\n        # {typ1} - {typ2}")
        try:
            if val2 is not None:
                print(f"        self.assertEqual(Qi({repr(val1)}, {repr(val2)}), {repr(Qi(val1, val2))})")
            else:
                print(f"        self.assertEqual(Qi({repr(val1)}), {repr(Qi(val1))})")
        except Exception as exc:
            print(f"        # <<< ERROR >>> {exc}")

The output of the cell below will be a complete test case.

To run it, first copy it into a separate cell and then run the cell.

In [18]:
header = """
from unittest import TestCase, TextTestRunner, defaultTestLoader

class TestQi(TestCase):

    def setUp(self) -> None:
        seed(42)

    def test_constructor(self):
        #-------------------
        # re type - im type
        #-------------------"""

in1 = [('str', '1/2'), ('float', 0.5), ('int', -3), ('Fraction', Fraction(1, 2))]
in2 = [('str', '3/4'), ('float', 0.75), ('int', 7), ('Fraction', Fraction(3, 8)), ('None', None)]
in3 = [('complex', (-1.5+2j)), ('Qi', Qi('1/2', '3/4')), ('Zi', Zi(3, -7))]
in4 = [('complex', (3-0.75j)), ('Qi', Qi('1/4', '3/4')), ('Zi', Zi(-2, 5))]

print(header)
print_Qi_unittests(in1, in2)
print_Qi_unittests(in3, in4)
print("\n# END OF FILE")


from unittest import TestCase, TextTestRunner, defaultTestLoader

class TestQi(TestCase):

    def setUp(self) -> None:
        seed(42)

    def test_constructor(self):
        #-------------------
        # re type - im type
        #-------------------

        # str - str
        self.assertEqual(Qi('1/2', '3/4'), Qi('1/2', '3/4'))

        # str - float
        self.assertEqual(Qi('1/2', 0.75), Qi('1/2', 0.75))

        # str - int
        self.assertEqual(Qi('1/2', 7), Qi('1/2', 7))

        # str - Fraction
        self.assertEqual(Qi('1/2', Fraction(3, 8)), Qi('1/2', Fraction(3, 8)))

        # str - None
        self.assertEqual(Qi('1/2'), Qi('1/2', None))

        # float - str
        self.assertEqual(Qi(0.5, '3/4'), Qi(0.5, '3/4'))

        # float - float
        self.assertEqual(Qi(0.5, 0.75), Qi(0.5, 0.75))

        # float - int
        self.assertEqual(Qi(0.5, 7), Qi(0.5, 7))

        # float - Fraction
        self.assertEqual(Qi(0.5, Fraction(3, 8)), Qi(0.5, Fractio

In [12]:
from unittest import TestCase, TextTestRunner, defaultTestLoader

class TestQi(TestCase):

    def setUp(self) -> None:
        seed(42)

    def test_constructor(self):
        #-------------------
        # re type - im type
        #-------------------

        # str - str
        self.assertEqual(Qi('1/2', '3/4'), Qi('1/2', '3/4'))

        # str - float
        self.assertEqual(Qi('1/2', 0.75), Qi('1/2', '3/4'))

        # str - int
        self.assertEqual(Qi('1/2', 7), Qi('1/2', '7'))

        # str - Fraction
        self.assertEqual(Qi('1/2', Fraction(3, 8)), Qi('1/2', '3/8'))

        # str - None
        self.assertEqual(Qi('1/2'), Qi('1/2', '0'))

        # float - str
        self.assertEqual(Qi(0.5, '3/4'), Qi('1/2', '3/4'))

        # float - float
        self.assertEqual(Qi(0.5, 0.75), Qi('1/2', '3/4'))

        # float - int
        self.assertEqual(Qi(0.5, 7), Qi('1/2', '7'))

        # float - Fraction
        self.assertEqual(Qi(0.5, Fraction(3, 8)), Qi('1/2', '3/8'))

        # float - None
        self.assertEqual(Qi(0.5), Qi('1/2', '0'))

        # int - str
        self.assertEqual(Qi(-3, '3/4'), Qi('-3', '3/4'))

        # int - float
        self.assertEqual(Qi(-3, 0.75), Qi('-3', '3/4'))

        # int - int
        self.assertEqual(Qi(-3, 7), Qi('-3', '7'))

        # int - Fraction
        self.assertEqual(Qi(-3, Fraction(3, 8)), Qi('-3', '3/8'))

        # int - None
        self.assertEqual(Qi(-3), Qi('-3', '0'))

        # Fraction - str
        self.assertEqual(Qi(Fraction(1, 2), '3/4'), Qi('1/2', '3/4'))

        # Fraction - float
        self.assertEqual(Qi(Fraction(1, 2), 0.75), Qi('1/2', '3/4'))

        # Fraction - int
        self.assertEqual(Qi(Fraction(1, 2), 7), Qi('1/2', '7'))

        # Fraction - Fraction
        self.assertEqual(Qi(Fraction(1, 2), Fraction(3, 8)), Qi('1/2', '3/8'))

        # Fraction - None
        self.assertEqual(Qi(Fraction(1, 2)), Qi('1/2', '0'))

        # complex - complex
        self.assertEqual(Qi((-1.5+2j), (3-0.75j)), Qi(Qi('-3/2', '2'), Qi('3', '-3/4')))

        # complex - Qi
        self.assertEqual(Qi((-1.5+2j), Qi('1/4', '3/4')), Qi(Qi('-3/2', '2'), Qi('1/4', '3/4')))

        # complex - Zi
        # <<< ERROR >>> Inputs incompatible: (-1.5+2j) and (-2+5j)

        # Qi - complex
        self.assertEqual(Qi(Qi('1/2', '3/4'), (3-0.75j)), Qi(Qi('1/2', '3/4'), Qi('3', '-3/4')))

        # Qi - Qi
        self.assertEqual(Qi(Qi('1/2', '3/4'), Qi('1/4', '3/4')), Qi(Qi('1/2', '3/4'), Qi('1/4', '3/4')))

        # Qi - Zi
        # <<< ERROR >>> Inputs incompatible: (1/2+3/4j) and (-2+5j)

        # Zi - complex
        # <<< ERROR >>> Inputs incompatible: (3-7j) and (3-0.75j)

        # Zi - Qi
        # <<< ERROR >>> Inputs incompatible: (3-7j) and (1/4+3/4j)

        # Zi - Zi
        self.assertEqual(Qi(Zi(3, -7), Zi(-2, 5)), Qi(Qi('3', '-7'), Qi('-2', '5')))

# END OF FILE

In [19]:
Qi('1/2', 0.75)

Qi('1/2', 0.75)

In [20]:
TextTestRunner(verbosity=2).run(defaultTestLoader.loadTestsFromTestCase(TestQi))

test_constructor (__main__.TestQi.test_constructor) ... FAIL

FAIL: test_constructor (__main__.TestQi.test_constructor)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/0y/ptqvq6x56tbdb9mz97r5tjk00000gn/T/ipykernel_17983/2764008250.py", line 17, in test_constructor
    self.assertEqual(Qi('1/2', 0.75), Qi('1/2', '3/4'))
AssertionError: Qi('1/2', 0.75) != Qi('1/2', '3/4')

----------------------------------------------------------------------
Ran 1 test in 0.005s

FAILED (failures=1)


<unittest.runner.TextTestResult run=1 errors=0 failures=1>

In [21]:
x1 = Qi(1/2, 3/4)
x1

Qi(0.5, 0.75)

In [22]:
x2 = Qi(Qi(1/2, 3/4), Qi(0.875, 3))
x2

Qi(Qi(0.5, 0.75), Qi(0.875, 3))

In [23]:
str(x1)

'NOT SUPPORTED'

In [24]:
str(x2)

'NOT SUPPORTED'

In [27]:
from abc import ABC, abstractmethod
from dataclasses import dataclass, field

class ValueHolder(ABC):
    """
    An abstract base class that holds two values, x and y.
    
    This class cannot be instantiated directly because of the @abstractmethod on __init__.
    Its purpose is to define the interface for child classes.
    """
    _x: field(init=False)
    _y: field(init=False)

    @abstractmethod
    def __init__(self, x, y):
        # The abstract __init__ enforces that child classes must implement
        # their own version, and they must call the superclass's __init__.
        # This approach ensures the parent is not instantiated.
        self._x = x
        self._y = y

    @property
    def x(self):
        return self._x
    
    @property
    def y(self):
        return self._y

@dataclass(frozen=True)
class ModifiedValues(ValueHolder):
    """
    An immutable child class that modifies the values from the parent.
    
    The frozen=True argument makes instances of this class immutable.
    The __post_init__ method is used to process the values after they
    have been set.
    """
    _x: any
    _y: any
    
    def __post_init__(self):
        """
        Post-initialization to alter values based on their types.
        Because the class is frozen, this method uses object.__setattr__
        to bypass the immutability restrictions.
        """
        # Create a mutable copy of the parent's values
        x = super().x
        y = super().y

        if isinstance(x, (int, float)):
            x += 10
        elif isinstance(x, str):
            x = x.upper()

        if isinstance(y, (int, float)):
            y -= 5
        elif isinstance(y, str):
            y = y.lower()

        # Reassign the values using object.__setattr__ to avoid a FrozenInstanceError
        object.__setattr__(self, '_x', x)
        object.__setattr__(self, '_y', y)

    def __init__(self, x, y):
        """
        Implementation of the abstract __init__ method.
        This simply forwards the values to the parent class.
        """
        super().__init__(x, y)

# --- Example Usage ---

# Test with integers
test1 = ModifiedValues(x=5, y=20)
print(f"Original values: x=5, y=20")
print(f"Modified values: x={test1.x}, y={test1.y}\n")  # Expected output: x=15, y=15

# Test with strings
test2 = ModifiedValues(x="hello", y="WORLD")
print(f"Original values: x='hello', y='WORLD'")
print(f"Modified values: x='{test2.x}', y='{test2.y}'\n") # Expected output: x='HELLO', y='world'

# Attempt to instantiate the abstract parent class (will raise a TypeError)
try:
    ValueHolder(1, 2)
except TypeError as e:
    print(f"Attempting to instantiate parent class caught: {e}\n")

# Attempt to change a value in the immutable child (will raise a FrozenInstanceError)
try:
    test1._x = 50
except dataclasses.FrozenInstanceError as e:
    print(f"Attempting to modify immutable child caught: {e}")



FrozenInstanceError: cannot assign to field '_x'